### Comparison between observed and calculated

In [ ]:
#!pip install scikit-learn

In [ ]:
#import the require packages
import flopy ## Org
import geopandas as gpd ## Org
import numpy as np ## Org
import matplotlib.pyplot as plt ## Org
from mf6Voronoi.utils import initiateOutputFolder ## Org
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score, mean_squared_error ## Org

In [ ]:
#define model name and model path
simName = 'mf6Sim' ## Org
modelName = 'mf6Model' ## Org
modelWs = 'modelFiles' ## Org

In [ ]:
#open the observation file out heads
obsOut = flopy.utils.observationfile.Mf6Obs(modelWs+'/'+modelName+'.ob_gw_out_head',isBinary=False) ## Org

In [ ]:
#create a dataframe for the observation points
totalObsDf = obsOut.get_dataframe().transpose() ## Org
totalObsDf = totalObsDf.rename(columns={totalObsDf.keys()[0]:'SimHead'}) ## Org
totalObsDf = totalObsDf.drop('totim',axis=0) ## Org
totalObsDf['ObsHead'] = 0 ## Org
totalObsDf['Easting'] = 0 ## Org
totalObsDf['Northing'] = 0 ## Org
totalObsDf.head() ## Org

In [ ]:
#open the observed data file
obsDf = gpd.read_file('shp/piezometers.shp') ## Org
obsDf.head() ## Org

In [ ]:
#look for the observed elevation and coordinates
for index, row in totalObsDf.iterrows(): ## Org
    for index2, row2 in obsDf.iterrows(): ## Org
        if index.lower() == row2.Name.lower(): ## Org
            totalObsDf.loc[index,'ObsHead'] = row2.waterHead ## Org
            totalObsDf.loc[index,'Easting'] = float(row2.geometry.x) ## Org
            totalObsDf.loc[index,'Northing'] = float(row2.geometry.y) ## Org

In [ ]:
#compute the residual and save csv file
#initiateOutputFolder('../Model/output')

totalObsDf = totalObsDf[totalObsDf['SimHead']>0] ## Org
totalObsDf['Residual'] = totalObsDf['SimHead'] - totalObsDf['ObsHead'] ## Org
totalObsDf.to_csv('output/HobDf_Out.csv') ## Org
totalObsDf.head() ## Org

In [ ]:
filterObsDf=totalObsDf.dropna() ## Org
filterObsDf ## Org

In [ ]:
#Create a basic scatter plot with Matplotlib
#A identity line was created based on the minumum and maximum observed value
#Points markers are colored by the residual and a residual colorbar is added to the figure
fig = plt.figure(figsize=(10,8)) ## Org

minMin = np.min([filterObsDf['SimHead'].min(),filterObsDf['ObsHead'].min()]) ## Org
maxMax = np.max([filterObsDf['SimHead'].max(),filterObsDf['ObsHead'].max()]) ## Org
meanSquareError = mean_squared_error(filterObsDf['ObsHead'],filterObsDf['SimHead']) ## Org
meanAbsoluteError = mean_absolute_error(filterObsDf['ObsHead'],filterObsDf['SimHead']) ## Org
r2score = r2_score(filterObsDf['ObsHead'],filterObsDf['SimHead']) ## Org
nrms = (meanSquareError**0.5)/(maxMax-minMin)*100 ## Org

x = np.linspace(filterObsDf['SimHead'].min()-5, filterObsDf['SimHead'].max()+5, 100) ## Org
plt.plot(x, x, linestyle='dashed') ## Org
plt.scatter(filterObsDf['ObsHead'],filterObsDf['SimHead'], marker='o', c=filterObsDf['Residual']) ## Org
for index, row in filterObsDf.iterrows(): ## Org
    if row['ObsHead'] > 0: ## Org
        plt.text(int(row['ObsHead']),int(row['SimHead']),index) ## Org

#cbar = plt.colorbar()
#cbar.set_label('Residual (m)', fontsize=14)

plt.grid() ## Org
plt.suptitle('Comparison Observed vs Simulated') ## Org
plt.title('''Nro of Points: %d 
    Mean square error: %.2f
    Mean absolute error: %.2f
    R2 score: %.2f
    NRMS(percentage): %.2f'''%(filterObsDf.shape[0],meanSquareError,meanAbsoluteError,r2score,nrms)) ## Org
plt.xlabel('Observed Head (m)', fontsize=14) ## Org
plt.ylabel('Simulated Head (m)', fontsize=14) ## Org
fig.tight_layout() ## Org